In [149]:
import pickle
from pathlib import Path
import copy

In [220]:
datatrain_pickle_path = Path('../reid/TransReID/data/aic21/datatrain.pickle')
# datatrain_pickle_path = Path('../reid/TransReID/data/aic21/aic21/')
tmp = pickle.load(datatrain_pickle_path.open('rb'))
test_paths = tmp['test_paths']
train_paths = tmp['train_paths']
# [len(test_path) for test_path in test_paths]

In [217]:
len(train_paths)

3092

## Preprocessing data
- remove parked bbox
- remove box with small
- calculate average_bbox for each track
- remove or boostrap bbbox to have average_bbox for each track

In [219]:
from scipy.spatial.distance import euclidean

# for track in test_paths:
#     for bbox in track
#         img_path = bbox[0]
#         tlwh = bbox[1]
        
def remove_parked_bbox(paths, park_thre=3):
    """
    Remove bounding box which have distance to the lastest bbox < park_thre
    """
    new_paths = []
    for track in paths:
        new_track = []
        central_point_previous = (0,0)
        for i,bbox in enumerate(track):
            t,l,w,h = bbox[1]
            central_point = ((t+w)/2, (l+h)/2)
            if (euclidean(central_point, central_point_previous) > park_thre) or (central_point_previous==(0,0)):
                new_track.append(bbox)
                central_point_previous = central_point
        new_paths.append(new_track)
    return new_paths

def remove_small_bbox(paths, h_threhold = 60):
    """
    Remove bbox with heigh <= 60
    """
    new_paths = []
    for track in paths:
        new_track = list(filter(lambda bbox: bbox[1][3]>h_threhold , track))
        new_paths.append(new_track)
    return new_paths
    
def get_num_ave_bbox(paths):
    n_bbox_ls = [len(track) for track in paths]
    num_ave_bbox = sum(n_bbox_ls)/len(n_bbox_ls)
    return int(num_ave_bbox)

def stem_track(paths, num_ave_bbox):
    """
    If len(track) > num_ave_bbox: rm bbox in order [0,-1,2,-3,4,-5....]
    If len(track) < num_ave_bbox: with n is ave idx, boostrap bbox in order [n, n+2, n-2, n+4, n-4.....]
    """
    def remove_bbox_(track, num_ave_bbox):
        len_track = len(track)
        n_remove = len(track) - num_ave_bbox
        rm_idx =[]
        for i in range(n_remove):
            if i == 0:
                idx = 0
            else:
                idx = -idx-1 if i%2!=0 else -idx+1
            rm_idx.append(idx)
            
        rm_idx = [idx if idx>=0 else len_track+idx for idx in rm_idx ]
#         print (rm_idx)
        track = [item for i,item in enumerate(track) if i not in rm_idx]
        if len(track) != num_ave_bbox:
            track = remove_bbox_(track, num_ave_bbox)
#         print(len(track))
        return track
    
    def insert_value_(track, item):
        idx = track.index(item)
        track.insert(idx, item)
        return track

    def boostrap_bbox_(track, num_ave_bbox):
        """
        
        """ 
#         print (f"boostrap_bbox_{len(track)}_{num_ave_bbox}")
        ## add all tracks
        
        n_mul = int(num_ave_bbox/len(track))-1
        track_ori = copy.deepcopy(track)
        if n_mul != 0:
            for _ in range(n_mul):
                for item in track_ori:
                    track = insert_value_(track, item)
#         print (f"n_mul_{n_mul}")
                
        # add each ele of tracks
        n_add = num_ave_bbox - len(track)
        add_items = []
#         idxs =[]
        i=0
        for _ in range(n_add):
            if i == 0:
                idx = int(len(track)/2)
                i+=1
            else:
                idx = idx + 2*i if i%2!=0 else idx - 2*i
                i+=1
            
            if idx not in range(len(track)):
                idx = int(len(track)/2)+1
                i=1
#             print(f'idx:{idx}')
#             idxs.append(idx)
            add_items.append(track[idx])
#         print(idxs)
        
        for item in add_items:
            track = insert_value_(track, item)
#         print(len(track))
        return track

    new_paths = []
    for track in paths:
        if len(track) == num_ave_bbox:
            new_paths.append(track)
        elif len(track) > num_ave_bbox:
            new_paths.append(remove_bbox_(track, num_ave_bbox))
        else:
            new_paths.append(boostrap_bbox_(track, num_ave_bbox))
            
    return new_paths


test_paths = remove_parked_bbox(test_paths, park_thre=3)
test_paths = remove_small_bbox(test_paths)
train_paths = remove_parked_bbox(train_paths)
train_paths = remove_small_bbox(train_paths, h_threhold=60)
num_ave_bbox = get_num_ave_bbox(test_paths+train_paths)
test_paths = stem_track(test_paths,num_ave_bbox)
train_paths = stem_track(train_paths,num_ave_bbox)

ZeroDivisionError: division by zero

In [208]:
bbox = train_paths[0][0]
bbox[0].name.split("_")[2]
# bbox

'1'

## Feed data to Dataloader

In [31]:
import torch.nn.functional as F
import torch
import einops

scor = [-0.2905, -0.1682, -0.1891, -0.1500, -0.1603,  0.0883,  0.1807, -0.1686,
         -0.1562, -0.1691,  0.4487, -0.1810,  0.5024,  0.1063,  0.3008,  0.0775,
          0.1598,  0.1831, -0.1367,  0.1138, -0.1547, -0.1772,  0.4121,  0.4697,
         -0.1648,  0.0598, -0.1825, -0.1661,  0.3770,  0.1521, -0.1799, -0.3059,
          0.3398, -0.2102, -0.1672,  0.1365, -0.1791, -0.1741, -0.1792, -0.1829,
          0.1117, -0.1720, -0.1444,  0.1785, -0.1736, -0.1639, -0.1794, -0.1876,
         -0.1472, -0.1681,  0.2678, -0.1722, -0.1472,  0.1373,  0.1926, -0.1534,
         -0.1680, -0.1669,  0.0482,  0.1140,  0.4932, -0.1470,  0.4561,  0.1307,
          0.0776,  0.1865, -0.1554, -0.2336,  0.1700, -0.1659,  0.4536, -0.1541,
         -0.1594, -0.1906, -0.1562, -0.1769, -0.1735,  0.2202,  0.1469, -0.1442,
          0.1591, -0.1658, -0.1605, -0.1779, -0.1271, -0.1791, -0.2869, -0.1648,
          0.0610,  0.0492, -0.1785, -0.2893, -0.1709, -0.1796, -0.1649, -0.1870,
         -0.1757, -0.1511, -0.1757, -0.1791,  0.1047, -0.1394, -0.1465,  0.2649,
          0.3894, -0.1508, -0.1841, -0.1534, -0.1699, -0.1547,  0.1722, -0.1741,
         -0.1628, -0.1613, -0.1941, -0.1697, -0.3188, -0.1720, -0.1837, -0.1637,
         -0.1470, -0.1484, -0.1558,  0.1356, -0.1655, -0.1632,  0.1790, -0.1641,
         -0.1586, -0.1753, -0.1698, -0.1804, -0.1605,  0.0878, -0.1646, -0.1447,
         -0.1462, -0.1707, -0.1609, -0.1702,  0.2021, -0.1854, -0.1747,  0.2240,
         -0.1348, -0.1663, -0.1526, -0.1766,  0.1927,  0.1774, -0.1647,  0.1558,
         -0.1808,  0.1729, -0.1664, -0.2100, -0.1737, -0.1913, -0.1628, -0.1710,
         -0.1492, -0.1669,  0.0510, -0.2527, -0.1720, -0.1730, -0.1763, -0.1703,
          0.0761, -0.1654, -0.1586,  0.1201,  0.1744,  0.1082,  0.0790,  0.1720,
          0.0698,  0.2319,  0.1483, -0.1593, -0.1721,  0.1511, -0.1581,  0.3689,
         -0.1643,  0.0198,  0.3345,  0.1136, -0.1874, -0.1633,  0.1614,  0.2000,
         -0.1625,  0.0534, -0.1616, -0.1851,  0.2524,  0.1589, -0.1658, -0.1658,
         -0.1809,  0.1355, -0.1724, -0.1588, -0.1849,  0.1390, -0.1558, -0.1504,
          0.1615, -0.1648,  0.1530, -0.1637, -0.2908, -0.1665, -0.1536, -0.1613,
          0.1866, -0.1805, -0.1760, -0.1604, -0.1582, -0.1990,  0.4421,  0.1617,
          0.1631,  0.1780, -0.1832,  0.1691,  0.0840, -0.1670, -0.1740, -0.1644,
         -0.1547, -0.1465,  0.0818,  0.4077,  0.1729, -0.1750, -0.1472, -0.1693,
          0.9673,  0.6504, -0.1877,  0.0381,  0.5122,  0.4006,  0.1547,  0.2073,
          0.2646,  0.0742,  0.1160,  0.0752, -0.1729,  0.1737, -0.1697,  0.1953,
          0.0492,  0.1740,  0.1827,  2.0332,  0.0757, -0.1708, -0.1770, -0.0459,
          0.4570,  0.1699, -0.1552, -0.1606,  0.0786, -0.0443,  0.1081,  0.8940,
          0.7080, -0.1678,  0.5039, -0.1923,  0.0964,  0.1141, -0.1921,  0.1025,
          0.1667, -0.0028, -0.1672,  0.4182, -0.0382,  0.1359,  0.0818,  0.1423,
         -0.1635, -0.1663, -0.1716, -0.1578, -0.1805,  0.1840,  0.5967, -0.1702,
          0.5537,  0.3533,  0.0617,  0.1805, -0.1382, -0.1809,  0.1758, -0.1705,
          0.4270,  0.0729,  0.1249, -0.1665,  0.5767, -0.1807,  0.1906,  0.1395,
          0.0708, -0.1587,  0.4731,  0.5303, -0.1829,  0.3953, -0.1752,  0.0963,
         -0.1442, -0.3657, -0.1697,  0.0246,  0.4260, -0.1741,  0.3855, -0.1477,
         -0.1782,  0.4409, -0.1814, -0.1477,  0.1691, -0.1611,  0.0913,  0.6968,
          0.1400,  0.1577, -0.1750,  0.1611, -0.1487, -0.1727, -0.1772,  0.4050,
         -0.2413,  0.1268,  0.0022,  0.7935, -0.1715, -0.1707,  0.8125,  0.1790,
         -0.1443,  0.7910, -0.1643,  0.0853,  0.1873,  0.4761,  0.0598, -0.1481,
         -0.1672, -0.1652,  0.0807,  0.0507, -0.2198, -0.1724, -0.1821, -0.1639,
          0.0713,  0.0812,  0.3789,  0.1980,  0.1530, -0.1731,  0.0705, -0.1710,
         -0.1638,  0.1458,  0.5107,  0.4895,  0.3000,  0.1722, -0.1694,  0.1538,
         -0.1790,  0.4539,  0.4185,  0.2067, -0.1532, -0.1514,  0.7476, -0.1581,
         -0.1561, -0.1772, -0.1443,  0.1833,  0.1710, -0.1749, -0.1580,  0.5044,
         -0.1736,  0.4478,  0.4429, -0.1863, -0.1793, -0.1525, -0.1603,  0.5791,
          0.0997, -0.1852,  0.1096, -0.1602,  0.9414,  0.9248, -0.1517,  0.7349,
         -0.1738,  0.1693,  0.1504,  0.4185, -0.1669, -0.1528, -0.1708,  0.1072,
          0.1471, -0.1814, -0.1685,  0.1842,  0.4282,  0.0948,  0.0738,  0.1959,
          0.5171, -0.1559, -0.1775,  0.1616, -0.1652,  0.0837, -0.1527, -0.1879,
         -0.1636, -0.1681, -0.1545,  0.3350, -0.1660,  0.0352,  0.4937,  0.3975,
          0.0214, -0.1636, -0.1757,  0.5161, -0.1665, -0.1760, -0.1768, -0.1644,
          0.0934,  0.1102, -0.1600, -0.1583,  0.0563, -0.3000, -0.0505, -0.1545,
          0.7734,  0.1685, -0.1545, -0.1465, -0.1879,  0.1724,  0.0964, -0.1499,
          0.0513,  0.4155, -0.1643, -0.1694, -0.1689, -0.3057, -0.1565, -0.1748,
          0.1592, -0.1725,  0.0974,  0.1843,  0.1582, -0.1641, -0.1740, -0.1716,
         -0.1799, -0.1631,  0.2864, -0.1641, -0.1776, -0.1681,  0.1498,  0.1501,
         -0.1951, -0.3713, -0.1559,  0.1210,  0.1632, -0.1639,  0.0905,  0.1958,
         -0.1689,  0.3892, -0.1586, -0.1708, -0.1787, -0.1744, -0.1765,  0.1625,
         -0.1035, -0.1848,  0.0547,  0.4209,  0.1722, -0.1788,  0.0378, -0.1746,
         -0.1710, -0.1575,  0.1058, -0.1599, -0.1746, -0.1721,  0.1761, -0.1720,
          0.1919, -0.1631,  0.0531, -0.1725,  0.1703,  0.1154, -0.1754, -0.1824,
         -0.1622,  0.3159, -0.1781, -0.1769, -0.1604,  0.1351, -0.1570, -0.1559,
         -0.1829, -0.0342, -0.1686, -0.1685,  0.2603, -0.1718,  0.0904, -0.1580,
         -0.1682, -0.1858, -0.1626, -0.1497, -0.0022, -0.1813, -0.1818, -0.1365,
          0.3491, -0.1521,  0.1654,  0.1508, -0.1764, -0.1443, -0.1636, -0.1886,
         -0.1638,  0.0910, -0.1857, -0.1366,  0.0980,  0.2040,  0.0868,  0.1663,
          0.1550, -0.1583,  0.2952, -0.1697, -0.1676, -0.1705, -0.1473, -0.1697,
         -0.1823, -0.1616,  0.1779,  0.1807, -0.1752,  0.1285,  0.4353, -0.1729,
         -0.1509, -0.1654,  0.1691, -0.1614, -0.1527, -0.1487, -0.1803, -0.1836,
          0.1853, -0.1548, -0.1501,  0.0803, -0.1769, -0.1573, -0.1855,  0.1019,
         -0.1461, -0.1846, -0.1541,  0.1713, -0.1559, -0.1738, -0.1809, -0.1881,
          0.1525, -0.1715, -0.1432,  0.4917,  0.0774,  0.0688, -0.1572,  0.0636,
         -0.1914,  0.1777, -0.1587, -0.1486,  0.1514, -0.1719,  0.0825, -0.1898,
         -0.1918,  0.0723,  0.1617, -0.1570,  0.2401, -0.1497, -0.1621,  0.3579,
          0.0302, -0.1809,  0.1616,  0.1814,  0.5684, -0.1733, -0.0502, -0.1527,
         -0.3113, -0.1724, -0.1708,  0.0945, -0.1804,  0.1410, -0.1653, -0.1395,
         -0.1802, -0.1505, -0.1678, -0.1343, -0.1517, -0.1726, -0.1428, -0.1844]

scor = torch.tensor(scor)
scor = einops.repeat(scor, 'a -> b a', b = 1)
target = torch.tensor([663])
target

tensor([663])

In [41]:
tids = set(range(10))
tid_renew = {tid: i for i, tid in enumerate(tids)}
tid_renew

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}

In [32]:
loss = F.cross_entropy(scor, target)
loss

tensor(6.7198)

In [36]:
[i for i in set(sorted(list(range(10)), reverse=True))]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]